# 7. Hipótese e Inferência

* A parte ciência de data science frequentemente envolve formar e testar hipóteses sobre nossos dados e os processos que o geram. 

### Teste Estatístico de Hipótese

* Com frequência queremos testar se uma determinada hipótese é verdadeira.
* Hipóteses são afirmações como "esta é uma moeda honesta" ou "os cientistas de dados preferem Python a R" que possam ser traduzidas em estatísticas sobre dados. 
* Essas estatísticas podem ser vistas como observações de variáveis aleatórias a partir de distribuições conhecidas. 
  * Permite que sejam feitas declarações sobre as premissas mais prováveis de serem corretas.


* Em uma configuração clássica, temos:
  * Hipótese nula $H_{0}$ que representa uma posição padrão e;
  * Alguma hipótese $H_{1}$ com a qual gostaríamos de compará-la.
  
  
* A partir disso, usamos estatística para decidir se $H_{0}$ é falsa ou não.

#### Exemplo: Lançar uma Moeda

* Imagine que temos uma moeda e queremos testar para confirmar se ela é honesta. 
  * $H_{0}$: "A moeda é honesta." ou possibildade de cair cara $p = 0,5$
  * $H_{1}$: $p \neq 0,5$
  
  
* Para o teste, uma moeda será lançada n vezes contando o número de caras X.
* Cada lançamento da moeda é uma tentativa de Bernoulli (Cap.06), e pode ser aproximada usando a distribuição normal.

In [1]:
# Funções Auxiliares do cap.06 que serão úteis para esse capítulo
import math
def normal_pdf(x, mu=0, sigma=1):
    sqrt_two_pi = math.sqrt(2*math.pi)
    return (math.exp(-(x-mu)**2/2/sigma**2)/(sqrt_two_pi*sigma))

# Função que pode ser usada para descobrir a probabilidade
def normal_cdf(x, mu=0,sigma=1):
    return (1 + math.erf((x - mu) / math.sqrt(2) / sigma)) / 2

def inverse_normal_cdf(p, mu=0, sigma=1, tolerance=0.00001):
    """find approximate inverse using binary search"""

    # if not standard, compute standard and rescale
    if mu != 0 or sigma != 1:
        return mu + sigma * inverse_normal_cdf(p, tolerance=tolerance)

    low_z, low_p = -10.0, 0            # normal_cdf(-10) is (very close to) 0
    hi_z,  hi_p  =  10.0, 1            # normal_cdf(10)  is (very close to) 1
    while hi_z - low_z > tolerance:
        mid_z = (low_z + hi_z) / 2     # consider the midpoint
        mid_p = normal_cdf(mid_z)      # and the cdf's value there
        if mid_p < p:
            # midpoint is still too low, search above it
            low_z, low_p = mid_z, mid_p
        elif mid_p > p:
            # midpoint is still too high, search below it
            hi_z, hi_p = mid_z, mid_p
        else:
            break

    return mid_z

In [2]:
# Aproximação da variável aleatória binomial
def normal_approximation_to_binomial(n, p):
    mu = p*n
    sigma = math.sqrt(p*(1-p)*n)
    return mu, sigma

* Como a variável aleatória segue uma distribuição normal, a função normal_cdf (cap.06) pode ser usada para descobrir a probabilidade dos valores resultantes serem internos ou externos a um determinado intervalo.

In [3]:
# o cdf normal é a probabilidade que a variável esteja abaixo de um limite
normal_probability_below = normal_cdf

# está acima do limite se não estiver abaixo
def normal_probability_above(lo, mu=0, sigma=1):
    return 1 - normal_cdf(lo, mu, sigma)

# está entre se for menor que hi e maior que low 
def normal_probability_between(lo, hi, mu=0, sigma=1):
    return normal_cdf(hi, mu, sigma)-normal_cdf(lo, mu, sigma)

# está fora se não estiver entre
def normal_probability_outside(lo, hi, mu=0, sigma=1):
    return 1 - normal_probability_between(lo, hi, mu, sigma)

* Podemos encontrar a região sem o intervalo específico em torno da média que contribui para o nível de probabilidade. 

In [4]:
def normal_upper_bound(probability, mu=0, sigma=1):
    return inverse_normal_cdf(probability, mu, sigma)

def normal_lower_bound(probability, mu=0, sigma=1):
    return inverse_normal_cdf(1-probability, mu, sigma)

def normal_two_sided_bound(probability, mu=0, sigma=1):
    tail_probability = (1-probability)/2
    upper_bound = normal_lower_bound(tail_probability, mu, sigma)
    lower_bound = normal_upper_bound(tail_probability, mu, sigma)
    return lower_bound, upper_bound

* Decidimos lançar a moeda 1000 vezes (n = 1000). 
* Se for uma moeda honesta, X deve ser distribuido normalente com média 500 e desvio padrão de 15,8:

In [5]:
mu_0, sigma_0 = normal_approximation_to_binomial(1000, 0.5)
print("Média: {}\nDesvio Padrão: {:.1f}".format(mu_0, sigma_0))

Média: 500.0
Desvio Padrão: 15.8


* Significância: limite máximo de falsos prositivos (rejeita H0 mesmo sendo verdadeiro), aqui para o exemplo 5%.
* Com isso, consideramos que o teste rejeita $H_{0}$ se X cai fora dos limites abaixo:

In [6]:
normal_two_sided_bound(0.95, mu_0, sigma_0)

(469.01026640487555, 530.9897335951244)

* Se H0 for verdadeiro, o teste apresentará o resultado correto em 19 de 20 vezes.
* Poder de um teste: probabilidade de não rejeitar $H_{0}$ mesmo sendo falso.
* Para medir esse procedimento, deve-se especificar o que realmente significa $H_{0}$ ser falso.
* Veremos o que acontece se p = 0,55, quando a moeda está inclinada a cara.
* Podemos calcular assim:

In [7]:
# 95% dos resultados baseados na premissa de que p = 0,5 
lo, hi = normal_two_sided_bound(0.95, mu_0, sigma_0)

# mi e sigma reais baseados em p = 0,55
mu_1, sigma_1 = normal_approximation_to_binomial(1000, 0.55) 

type_2_probability = normal_probability_between(lo, hi, mu_1, sigma_1)
power = 1 - type_2_probability
print("Power: {:.3}".format(power))

Power: 0.887


* Queremos um teste que rejeitasse $H_{0}$ quando X fosse muito maior que 50, mas não quando x for menor.
* Para tal devemos encontrar o corte abaixo de 95% em que a probabilidade ficaria: 

In [8]:
hi = normal_upper_bound(0.95, mu_0, sigma_0)
print("Máximo: {:.0f}".format(hi))

type_2_probability = normal_probability_below(hi, mu_1, sigma_1)
power = 1 - type_2_probability
print("Power: {:.3}".format(power))

Máximo: 526
Power: 0.936


* Esse teste não rejeita $H_{0}$ quando X está abaixo de 469 (Improvável de acontecer se $H_{1}$ for verdadeiro) e rejeita $H_{0}$ quando X está entre 526 e 531 (provável de acontecer se $H_{1}$ for verdadeiro).

### p-Values

* Uma maneira alternativa de pensar no teste anterior é usando p-values. 
* Em vez de escolher limites, calcula-se a probabilidade (sendo $H_{0}$ verdadeiro) que podemos ver um valor ao menos tão extremo quanto ao que realmente observamos.

* Para o teste da moeda honesta: 

In [9]:
def two_sided_p_value(x, mu=0,sigma=1):
    if x >= mu:
        return 2*normal_probability_above(x, mu, sigma)
    else:
        return 2*normal_probability_below(x, mu, sigma)
    
# Para 530 caras, computaríamos:
two_sided_p_value(529.5,mu_0,sigma_0) # Correção de continuidade, motivo pelo qual utilizamos 529.5
                                       # ao invés de 530, trata-se da melhor estimativa de ver ao
                                        # ao menos 530 caras.

0.06207721579598835

* Abaixo uma simulação para para comprovar a relevância dessa estimativa:

In [10]:
import random
extreme_value_count = 0
for _ in range(100000):
    num_heads = sum(1 if random.random() < 0.5 else 0 # contagem do # de caras
                    for _ in range(1000)) # em 1000 lançamentos
    if num_heads >= 530 or num_heads <= 470: # e contagem da frequência
        extreme_value_count += 1 # que # é 'extrema'

print (extreme_value_count / 100000)

0.06181


* Se p-value for maior que a significância de 5%, não rejeitamos a hipótese $H_{0}$.

In [13]:
# Se tivéssemos 532 caras
t1 = two_sided_p_value(531.5, mu_0, sigma_0) # < 0.05, rejeitado

upper_p_value = normal_probability_above
lower_p_value = normal_probability_below

# Se tivéssemos 525 caras
t2 = upper_p_value(524.5, mu_0, sigma_0) # > 0.05, aceito

# Se tivéssemos 527 caras
t3 = upper_p_value(526.5, mu_0, sigma_0) # < 0.05, rejeitado

print(t1, t2, t3)

0.046345287837786575 0.06062885772582072 0.04686839508859242


### Intervalos de Confiança

* Uma terceira abordagem ao problema abordado é utilizar intervalos de confiança em torno do valor observado.
* Quão confiantes podemos ser em uma estimativa?
  * Se soubessemos exatamente o valor de p, seria fácil utilizar o teorema do limite central.
  * Como p não é conhecido, será necessário utilizar a estimativa com o teorema do limite central para obter o intervalo. Não é totalmente confiável, mas é popularmente adotado. 
  * Para obter tal estimativa, dizemos estar "95% confiantes" de obter o valor de p.

In [14]:
# Observando 525 caras
p_hat = 525/1000
mu = p_hat
sigma = math.sqrt(p_hat * (1 - p_hat) / 1000)

normal_two_sided_bound(0.95, mu, sigma) # Não é viciada, cai dentro do intervalo de confiança.

(0.4940490278129096, 0.5559509721870904)

In [15]:
# Observando 525 caras
p_hat = 540/1000
mu = p_hat
sigma = math.sqrt(p_hat * (1 - p_hat) / 1000)

normal_two_sided_bound(0.95, mu, sigma) # Viciada, fora do intervalo de confiança.

(0.5091095927295919, 0.5708904072704082)

### P-Hacking

* Um procedimento que rejeita a hipótese nula erroneamente somente 5% das vezes vai — por definição — rejeitar erroneamente 5% das vezes a hipótese nula: 

In [16]:
def run_experiment():
    return [random.random() < 0.5 for _ in range(1000)]

def reject_fainess(experiment):
    num_heads = len([flip for flip in experiment if flip])
    return num_heads < 469 or num_heads > 531

random.seed(0)
experiments = [run_experiment() for _ in range(1000)]
num_rejections = len([experiment
                     for experiment in experiments
                     if reject_fainess(experiment)])

print("Número de Rejeições: {}".format(num_rejections))

Número de Rejeições: 46


* O que isso quer dizer é que você está tentando encontrar resultados "significativos" e geralmente você consegue.
* Teste hipóteses suficientes contra o seu conjunto de dados e um deles, certamente, parecerá significante.
* Remova os valores discrepantes certos, e será provável conseguir seu p-value abaixo de 0,05.
* Uma consequência da “inferência a partir da estrutura dos p-values”.

#### Executando um Teste A/B

Uma de nossas responsabilidades iniciais na DataSciencester é testar a otimização, um eufemismo para tentar fazer com que as pessoas cliquem nos anúncios. Um de seus anunciantes desenvolveu uma bebida energética voltada para os cientistas de dados, e o vice-presidente de Publicidade quer a sua ajuda para escolher entre a propaganda A (“bom sabor!”) ou propaganda B (“menos polarização!”).

Por ser um cientista, você decide executar um experimento mostrando aos visitantes do site uma das duas propagandas e registrando quantas pessoas clicam em cada um.

* Se 990 de 1000 visualizadores clicam no anúncio A enquanto 10 de 1000 visualizadores clicam no anúncio B. Chegamos a conclusão de que A é melhor que B. 
* Mas, e se as diferenças não forem tão óbvias?? 
  * Inferência Estatística
  
* Digamos que $N_{A}$ pessoas vejam o anúncio A e $n_{A}$ cliquem nele.
  * $p_{A}$ é a probabilidade de alguém clicar no anúncio A.
  * Então, sabemos que $\frac{n_{A}}{N_{A}}$ tem média $p_{A}$ e desvio padrão $\sigma_{A} = \sqrt{\frac{p_{A}(1 - p_{A})}{N_{A}}}$
  
* O mesmo para B, então:
  * $\frac{n_{B}}{N_{B}}$ tem média $p_{B}$ e desvio padrão $\sigma_{B} = \sqrt{\frac{p_{B}(1 - p_{B})}{N_{B}}}$

In [17]:
def estimated_parameters(N, n):
    p = n/N
    sigma = math.sqrt(p*(1-p)/N)
    return p, sigma

* Sendo A e B são independentes, então suas diferenças também deveriam ser normais com a média $p_{B} - p_{A}$ e o desvio padrão $\sqrt{\sigma^2_{A} + \sigma^2_{B}}$.
* Podemos testar a hipótese nula de que $p_{A}$ e $p_{B}$ são a mesma ($p_{B} - p_{A}$ é zero) usando a estatística:

In [18]:
def a_b_test_statistic(N_A, n_A, N_B, n_B):
    p_A, sigma_A = estimated_parameters(N_A, n_A)
    p_B, sigma_B = estimated_parameters(N_B, n_B)
    return (p_B - p_A)/math.sqrt(sigma_A**2 + sigma_B**2)

In [23]:
# "Bom sabor" recebe 200 cliques de 1000 views e "menos polarização" receba 180 de 1000 views
z = a_b_test_statistic(1000, 200, 1000, 180) 
print("Estatística: {:.2f}".format(z))
print("Probabilidade de ver tal diferença: {:.3f}".format(two_sided_p_value(z)))

Estatística: -1.14
Probabilidade de ver tal diferença: 0.254


In [24]:
# "menos polarização" receba 150 cliques
z = a_b_test_statistic(1000, 200, 1000, 150) 
print("Estatística: {:.2f}".format(z))
print("Probabilidade de ver tal diferença: {:.3f}".format(two_sided_p_value(z)))

Estatística: -2.95
Probabilidade de ver tal diferença: 0.003


### Inferência Bayesiana

* Uma alternativa para a inferência envolve tratar os parâmetros desconhecidos como variáveis aleatórias. 
* Deve-se começar com uma distribuição anterior para os parâmetros e usar os dados e o Teorema de Bayes para receber uma atualização da distribuição posterior para os parâmetros.

In [25]:
# Quando o parâmetro desconhecido é uma probabilidade
# Utilizamos uma anterior a partir da distribuição beta
# Coloca todas a probabilidades entre 0 e 1
def B(alpha, beta): 
    return math.gamma(alpha)*math.gamma(beta)/math.gamma(alpha+beta)

def beta_pdf(x, alpha, beta):
    if x < 0 or x > 1:
        return 0
    return x**(alpha-1)*(1-x)**(beta-1)/B(alpha,beta)

* O Teorema de Bayes diz que a distribuição posterior para p é novamente ua distribuição Beta mas com alpha + h (caras) e beta + t (coroas) (Para o exemplo do lançamento da moeda).
* O uso da inferência Bayesiana para testar hipóteses é considerado um pouco controverso — em parte porque sua matemática pode se tornar complicada e, em parte, por causa da natureza subjetiva de se escolher uma anterior.